In [1]:
import numpy as np
from scipy.optimize import minimize
from pseudo_code_from_Max import generate_system_mueller_matrix, update_systemMM, generate_measurement, perform_optimization

# Using previous fit
previous_fit = np.load("scipy_minimize_20230914_675nm_restrictive_HWP_and_IMR.npy")

# Starting guess values
theta_pol = previous_fit[0]
delta_HWP = previous_fit[1] # Add your actual delta_HWP value
offset_HWP = previous_fit[2] # Add your actual offset_HWP value
delta_derot = previous_fit[3] # Add your actual delta_derot value
offset_derot = previous_fit[4] # Add your actual offset_derot value
delta_opts = previous_fit[5] # Add your actual delta_opts value
epsilon_opts = previous_fit[6] # Add your actual epsilon_opts value
rot_opts = previous_fit[7] # Add your actual rot_opts value
delta_FLC = previous_fit[8] # Add your actual delta_FLC value
rot_FLC = previous_fit[9] # Add your actual rot_FLC value
em_gain = previous_fit[10] # Add your actual em_gain value

# Define the instrument configuration as a system dictionary
system_dict = {
    "components": {
        "parang_rot": {
            "type": "Rotator",
            "properties": {"pa": 0},
            "tag": "on-sky",
        },
        "m3": {
            "type": "DiattenuatorRetarder",
            "properties": {"phi": 2 * np.pi * 0, "epsilon": 0.03},
            "tag": "on-sky",
        },
        "alt_rot": {
            "type": "Rotator",
            "properties": {"pa": 0},
            "tag": "on-sky",
        },
        "lp": {
            "type": "LinearPolarizer",
            "properties": {"theta": theta_pol},
            "tag": "internal",
        },
        "hwp": {
            "type": "Retarder",
            "properties": {"phi": 2 * np.pi * delta_HWP, "delta_theta": offset_HWP},
            "tag": "internal",
        },
        "image_rotator": {
            "type": "Retarder",
            "properties": {"phi": 2 * np.pi * delta_derot, "delta_theta": offset_derot},
            "tag": "internal",
        },
        "optics": {
            "type": "DiattenuatorRetarder",
            "properties": {"phi": 2 * np.pi * rot_opts, "epsilon": epsilon_opts},
            "tag": "on-sky",
        },
        "flc": {
            "type": "Retarder",
            "properties": {"phi": 2 * np.pi * delta_FLC, "theta": rot_FLC},
            "tag": "internal",
        },
        "dichroic": {
            "type": "Retarder",
            "properties": {"phi": 2 * np.pi * 0, "theta": 0},
            "tag": "internal",
        },
        "wollaston": {
            "type": "WollastonPrism",
            "properties": {"beam": "o", "transmission_ratio": em_gain},
            "tag": "internal",
        },
    },
    "order": [
        "parang_rot", "m3", "alt_rot", "lp", "hwp",
        "image_rotator", "optics", "flc", "dichroic", "wollaston"
    ],
}

# Starting guess for parameters (aligned with bounds)
starting_guess = np.array([
    0,  # dichroic_phi
    0 # dichroic_theta
])

# Bounds for the parameters
bounds = [
    (0, 1),          # dichroic_phi
    (-180, 180),       # dichroic_theta
]

# Example data and errors for testing
data = [np.array([1, 0.5, -0.3, 0])]  # Example observed Stokes vectors
errors = [np.array([0.01, 0.01, 0.01, 0.01])]  # Example measurement errors

# Define system parameters in the expected nested format
system_parameters = [
    ["dichroic", "phi"],
    ["dichroic", "theta"]
]

# Perform optimization
result = perform_optimization(starting_guess, bounds, [system_dict], data, 
    errors, system_parameters, S_in = [1, 1, 0, 0])

# Print the results
print("Optimization Result:")
print("Success:", result.success)
print("Optimal Parameters:", result.x)
print("Final Objective Value (Chi-Squared):", result.fun)


Optimization Result:
Success: True
Optimal Parameters: [ 0.451 -2.642  0.32   0.036]
Final Objective Value (Chi-Squared): 6753.124999999999


In [ ]:
[ 1.53307627  0.46163077 -2.3687464   0.32815156 -4.99600063 -0.20433489
  0.02210595 18.49730849  0.52007057  4.99645591  1.19152714]

# Using JSON Files

In [3]:
import numpy as np
from scipy.optimize import minimize
from pseudo_code_from_Max import generate_system_mueller_matrix, update_systemMM, generate_measurement
import json

# Input: Specify the wavelength dynamically
wavelength = "670"  # Example: Replace with the desired wavelength

# Construct file paths dynamically based on the wavelength
json_file_path = f"instrument_configurations_{wavelength}.json"
double_diffs_path = f"double_diffs_{wavelength}.npy"
double_sums_path = f"double_sums_{wavelength}.npy"
double_diff_stds_path = f"double_diff_stds_{wavelength}.npy"
double_sum_stds_path = f"double_sum_stds_{wavelength}.npy"

# Load the saved JSON file for the given wavelength
with open(json_file_path, "r") as json_file:
    configurations = json.load(json_file)

# Load the arrays
double_diffs = np.load(double_diffs_path)
double_sums = np.load(double_sums_path)
double_diff_stds = np.load(double_diff_stds_path)
double_sum_stds = np.load(double_sum_stds_path)

# Print lengths and shapes
print(f"Length of double_diffs: {len(double_diffs)}, Shape: {double_diffs.shape}")
print(f"Length of double_sums: {len(double_sums)}, Shape: {double_sums.shape}")
print(f"Length of configurations: {len(configurations)}")

# Use previous fit values as a starting guess
previous_fit = np.load(f"scipy_minimize_20230914_{wavelength}nm_restrictive_HWP_and_IMR.npy")

# Extract starting parameters
theta_pol = previous_fit[0]
delta_HWP = previous_fit[1]
offset_HWP = previous_fit[2]
delta_derot = previous_fit[3]
offset_derot = previous_fit[4]
delta_opts = previous_fit[5]
epsilon_opts = previous_fit[6]
rot_opts = previous_fit[7]
delta_FLC = previous_fit[8]
rot_FLC = previous_fit[9]
em_gain = previous_fit[10]

# Starting guess for new parameters
starting_guess = np.array([
    0,  # dichroic_phi
    0   # dichroic_theta
])

# Bounds for the parameters
bounds = [
    (0, 1),         # dichroic_phi
    (-180, 180),    # dichroic_theta
]

# Define system parameters to optimize
system_parameters = [
    ["dichroic", "phi"],
    ["dichroic", "theta"]
]

# Define the objective function
def objective_function(params):
    """
    Objective function to minimize: chi-squared between model and observed data.
    """

    chi_squared = 0

    # Update dichroic parameters
    dichroic_phi, dichroic_theta = params

    for idx, config in enumerate(configurations):
        # Copy base configuration for modifications
        updated_system_dict = config.copy()
        updated_system_dict["components"]["dichroic"]["properties"]["phi"] = dichroic_phi
        updated_system_dict["components"]["dichroic"]["properties"]["theta"] = dichroic_theta

        # Generate Mueller matrices for different flux cases
        updated_system_dict["components"]["wollaston"]["properties"]["beam"] = "o"
        updated_system_dict["components"]["flc"]["properties"]["phi"] = 0  # FLC state A
        system_mm_FL1 = generate_system_mueller_matrix(updated_system_dict)

        updated_system_dict["components"]["wollaston"]["properties"]["beam"] = "e"
        system_mm_FR1 = generate_system_mueller_matrix(updated_system_dict)

        updated_system_dict["components"]["flc"]["properties"]["phi"] = np.pi / 4  # FLC state B
        updated_system_dict["components"]["wollaston"]["properties"]["beam"] = "o"
        system_mm_FL2 = generate_system_mueller_matrix(updated_system_dict)

        updated_system_dict["components"]["wollaston"]["properties"]["beam"] = "e"
        system_mm_FR2 = generate_system_mueller_matrix(updated_system_dict)

        # Generate simulated Stokes measurements for each configuration
        S_in = [1, 1, 0, 0]  # Example: unpolarized + equal linear polarization
        FL1 = generate_measurement(system_mm_FL1, S_in)[0]
        FR1 = generate_measurement(system_mm_FR1, S_in)[0]
        FL2 = generate_measurement(system_mm_FL2, S_in)[0]
        FR2 = generate_measurement(system_mm_FR2, S_in)[0]

        # Compute double difference (D) and double sum (S) using equations
        model_double_diff = ((FL1 - FR1) - (FL2 - FR2)) / ((FL1 + FR1) + (FL2 + FR2))
        model_double_sum = ((FL1 - FR1) + (FL2 - FR2)) / ((FL1 + FR1) + (FL2 + FR2))

        # Extract observed values
        observed_double_diff = double_diffs[idx]
        observed_double_sum = double_sums[idx]
        observed_double_diff_std = double_diff_stds[idx]
        observed_double_sum_std = double_sum_stds[idx]

        # Compute chi-squared contribution
        chi_squared += ((model_double_diff - observed_double_diff) / observed_double_diff_std) ** 2
        chi_squared += ((model_double_sum - observed_double_sum) / observed_double_sum_std) ** 2

    return chi_squared

# Perform the optimization
result = minimize(
    objective_function,
    starting_guess,
    bounds=bounds,
    method="Nelder-Mead"
)

# Print the optimization results
print("Optimization Result:")
print("Success:", result.success)
print("Optimal Parameters:", result.x)
print("Final Objective Value (Chi-Squared):", result.fun)

Length of double_diffs: 80, Shape: (80,)
Length of double_sums: 80, Shape: (80,)
Length of configurations: 80
Optimization Result:
Success: True
Optimal Parameters: [0. 0.]
Final Objective Value (Chi-Squared): 439896.7616672894
